<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import pandas as pd
import time

## 1. Download and Explore Dataset

#### Load and explore the data

In [4]:
link = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = urlopen(link)
html_file = f.read()
soup = BeautifulSoup(html_file, 'lxml')
#print(soup.prettify())
table_data = [[cell.text for cell in row("td")]
                         for row in BeautifulSoup(html_file)("tr")]
df = pd.read_json(json.dumps(table_data))
toronto_data = df[[0, 1, 2]]
toronto_data.drop(0, inplace=True)
toronto_data.dropna(inplace=True)
toronto_data.head()

D:\Python\Python36\Lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
D:\Python\Python36\Lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,0,1,2
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


#### rename columns names

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an dataframe.

In [5]:
# define the dataframe columns
toronto_data.rename(columns={0: 'PostalCode',1: 'Borough', 2: 'Neighborhood'}, inplace=True)
toronto_data.head()

D:\Python\Python36\Lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


#### explore "Borough"  values

In [6]:
toronto_data["Borough"].value_counts()

Not assigned        77
Etobicoke           44
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
B\n                  1
NL\n                 1
NS\n                 1
Queen's Park         1
Name: Borough, dtype: int64

#### drop [Not assigned, \n, NS\n, NL\n, B\n]  rows in 'Borough'

In [7]:
# Delete these row indexes from dataFrame
toronto_data.drop(toronto_data[toronto_data['Borough'] == 'Not assigned'].index , inplace=True)
toronto_data.drop(toronto_data[toronto_data['Borough'] == '\n' ].index , inplace=True)
toronto_data.drop(toronto_data[toronto_data['Borough'] == 'NS\n' ].index , inplace=True)
toronto_data.drop(toronto_data[toronto_data['Borough'] == 'NL\n' ].index , inplace=True)
toronto_data.drop(toronto_data[toronto_data['Borough'] == 'B\n' ].index , inplace=True)
toronto_data.reset_index(drop=True, inplace=True)

#### clean \n in "Neighborhood"

In [8]:
def clean_neighborhood(val):
    try:
        res = val.split('\n')[0]
    except AttributeError:
        res = val
    return res

In [9]:
neighborhood = toronto_data["Neighborhood"]
neighborhood = toronto_data["Neighborhood"].apply(clean_neighborhood)
neighborhood.head()
toronto_data["Neighborhood"] = neighborhood
try:
    toronto_data.drop("index", axis=1, inplace=True)
    pass
except KeyError:
    pass

D:\Python\Python36\Lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


#### quickly examine the resulting dataframe.

In [10]:
#toronto_data.shape
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [11]:
toronto_data.shape

(210, 3)

#### replase 'Not assigned' Neighborhood by Borough

In [12]:
def replase_neighborhood(row):
    if  "Not assigned" in row[2]:
        row[2]=row[1]
    return row

In [13]:
tmp_data = toronto_data.apply(replase_neighborhood, axis=1)
tmp_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### generate dataframe with combined Neighborhood by PostalCode and Borough

In [14]:
tmp_grouped = tmp_data.groupby(['PostalCode', 'Borough'], as_index=False)

In [15]:
k = list(tmp_grouped.groups.keys())
v = list(tmp_grouped.groups.values())
PostalCode_list = []
Borough_list = []
Neighborhood_list = []
for j in range(len(v)):
    tmp_str =""
    for i in v[j]:
        tmp_str = tmp_str + tmp_data.at[i, 'Neighborhood'] + ", "
        #print(tmp_data.at[i, 'Neighborhood'])
    tmp_str = tmp_str[0:-2]
    PostalCode_list.append(k[j][0])
    Borough_list.append(k[j][1])
    Neighborhood_list.append(tmp_str)
    
strange_df = pd.DataFrame({'PostalCode':PostalCode_list, 'Borough': Borough_list, 'Neighborhood':Neighborhood_list})
strange_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
strange_df.shape

(103, 3)